In [1]:
%pip install selenium
%pip install webdriver-manager
%pip install pandas
%pip install lxml
%pip install openpyxl
%pip install xlrd

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Célula 1: Importações e Configurações Iniciais
import os
import zipfile
import pandas as pd
import logging
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from io import StringIO
from datetime import datetime
import time

# Configuração de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Configuração das pastas
DOWNLOADS_FOLDER = "downloads"
BALANCOS_FOLDER = "balancos"
BALANCOS_XLSX_FOLDER = "balancos_xlsx"
BALANCOS_CONCATENADOS_FOLDER = "balancos_concatenados"
BALANCOS_DEFINITIVO_FOLDER = "balancos_definitivos"

# Criar pastas se não existirem
for folder in [DOWNLOADS_FOLDER, BALANCOS_FOLDER, BALANCOS_XLSX_FOLDER, BALANCOS_CONCATENADOS_FOLDER, BALANCOS_DEFINITIVO_FOLDER]:
    os.makedirs(folder, exist_ok=True)

In [4]:
# Célula 2: Configuração do WebDriver e Extração da Tabela de Resultados
# Configuração do WebDriver em modo headless
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
driver = webdriver.Chrome(options=options)

# URL da página de resultados
url_resultado = "https://www.fundamentus.com.br/resultado.php"

# Função para obter a tabela HTML
def obter_tabela_html(url, xpath):
    driver.get(url)
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    elemento = driver.find_element("xpath", xpath)
    return elemento.get_attribute("outerHTML")

# Obter a tabela HTML
html_tabela = obter_tabela_html(url_resultado, "/html/body/div[1]/div[2]/table")

# Converter a tabela HTML em DataFrame
df_dados_financeiros = pd.read_html(StringIO(html_tabela), decimal=",", thousands=".")[0]
df_dados_financeiros = df_dados_financeiros.sort_values(by="Papel", ascending=True).reset_index(drop=True)

# Exibir o DataFrame
print("Tabela com os dados financeiros:")
display(df_dados_financeiros)

# Lista de empresas
lista_de_empresas = df_dados_financeiros["Papel"].tolist()
print("Quantidade de empresas:", len(lista_de_empresas))

# Fechar o WebDriver
driver.quit()

Tabela com os dados financeiros:


,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,...,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,AALR3,15.00,-8.12,1.63,1.492,"0,00%",0.659,-11.41,34.91,-1.76,...,14.72,"4,28%","-17,52%",0.78,"2,21%","-20,06%",789446.0,1.089040e+09,0.77,"5,49%"
1,ABCB3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","14,96%",0.0,6.235070e+09,0.00,"13,50%"
2,ABCB4,23.55,6.18,0.92,0.000,"6,38%",0.000,0.00,0.00,0.00,...,0.00,"0,00%","0,00%",0.00,"0,00%","14,96%",16450400.0,6.235070e+09,0.00,"13,50%"
3,ABEV3,13.08,14.32,2.16,2.556,"5,58%",1.437,55.73,11.93,-22.68,...,8.28,"21,43%","18,36%",1.11,"15,82%","15,11%",340537000.0,9.526280e+10,0.04,"11,56%"
4,ABYA3,4.91,-214.80,1.76,2.055,"0,00%",0.527,1.98,19.96,-2.75,...,33.67,"10,29%","-0,96%",2.09,"2,78%","-0,82%",0.0,2.920600e+08,1.31,"16,41%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,WLMM4,36.00,9.75,1.78,0.429,"3,94%",1.215,3.24,7.36,4.05,...,7.44,"5,83%","4,40%",2.56,"19,69%","18,21%",23373.3,7.385320e+08,0.25,"28,41%"
983,WMBY3,25.39,-19.30,2.87,0.836,"0,00%",0.182,1.20,8.62,-1.50,...,25.02,"9,70%","-7,05%",1.44,"2,39%","-14,86%",0.0,2.124390e+08,6.51,"-14,48%"
984,WSON33,67.00,8.07,0.98,1.067,"0,00%",0.400,13.41,2.42,-0.89,...,5.08,"44,14%","13,58%",1.26,"18,36%","12,17%",0.0,2.148530e+09,1.21,"5,23%"
985,YDUQ3,10.31,21.77,1.01,0.601,"2,65%",0.339,3.63,4.03,-0.78,...,4.81,"14,91%","2,80%",1.68,"9,27%","4,66%",42661200.0,3.142670e+09,1.66,"9,76%"


Quantidade de empresas: 987


In [5]:
df_dados_financeiros.to_excel("dados_financeiros.xlsx", index=False)

In [6]:
# Célula 3: Funções Auxiliares
def obter_trimestre_anterior():
    mes_atual = datetime.now().month
    ano_atual = datetime.now().year
    if mes_atual <= 3:
        return f"Q4-{ano_atual-1}"
    elif mes_atual <= 6:
        return f"Q1-{ano_atual}"
    elif mes_atual <= 9:
        return f"Q2-{ano_atual}"
    else:
        return f"Q3-{ano_atual}"

trimestre_anterior = obter_trimestre_anterior()
pasta_trimestre = os.path.join(DOWNLOADS_FOLDER, trimestre_anterior)
os.makedirs(pasta_trimestre, exist_ok=True)

In [7]:
# Célula 4: Download dos Arquivos
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": os.path.join(os.getcwd(), pasta_trimestre),
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,   
}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(options=options)

def baixar_e_renomear_arquivo(papel):
    arquivo_destino = os.path.join(pasta_trimestre, f"{papel}.zip")
    if os.path.isfile(arquivo_destino):
        logging.info(f"Arquivo {arquivo_destino} já existe. Pulando o papel {papel}.")
        return

    url_papeis = f"https://www.fundamentus.com.br/balancos.php?papel={papel}&interface=mobile"
    driver.get(url_papeis)

    try:
        time.sleep(2)
        botao_download = WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, "a.bt-baixar"))
        )
        botao_download.click()

        # Esperar o download começar
        for _ in range(10):
            time.sleep(5)
            if any(f.startswith("bal_") for f in os.listdir(pasta_trimestre)):
                break

        novo_arquivo = next((f for f in os.listdir(pasta_trimestre) if f.startswith("bal_")), None)

        if novo_arquivo:
            caminho_arquivo = os.path.join(pasta_trimestre, novo_arquivo)
            os.rename(caminho_arquivo, arquivo_destino)
            logging.info(f"Arquivo {arquivo_destino} baixado e renomeado com sucesso!")
        else:
            logging.error(f"Erro ao encontrar o arquivo baixado para {papel}")

    except Exception as e:
        logging.error(f"Erro ao baixar arquivo para {papel}: {e}")

# Baixar e renomear arquivos para cada empresa
for papel in lista_de_empresas:
    baixar_e_renomear_arquivo(papel)

# Fechar o WebDriver
driver.quit()

2024-08-24 18:06:58,300 - INFO - Arquivo downloads\Q2-2024\AALR3.zip já existe. Pulando o papel AALR3.
2024-08-24 18:06:58,304 - INFO - Arquivo downloads\Q2-2024\ABCB3.zip já existe. Pulando o papel ABCB3.
2024-08-24 18:06:58,305 - INFO - Arquivo downloads\Q2-2024\ABCB4.zip já existe. Pulando o papel ABCB4.
2024-08-24 18:06:58,309 - INFO - Arquivo downloads\Q2-2024\ABEV3.zip já existe. Pulando o papel ABEV3.
2024-08-24 18:06:58,313 - INFO - Arquivo downloads\Q2-2024\ABYA3.zip já existe. Pulando o papel ABYA3.
2024-08-24 18:06:58,316 - INFO - Arquivo downloads\Q2-2024\ACES3.zip já existe. Pulando o papel ACES3.
2024-08-24 18:06:58,319 - INFO - Arquivo downloads\Q2-2024\ACES4.zip já existe. Pulando o papel ACES4.
2024-08-24 18:06:58,321 - INFO - Arquivo downloads\Q2-2024\ADHM3.zip já existe. Pulando o papel ADHM3.
2024-08-24 18:06:58,323 - INFO - Arquivo downloads\Q2-2024\AEDU11.zip já existe. Pulando o papel AEDU11.
2024-08-24 18:06:58,324 - INFO - Arquivo downloads\Q2-2024\AEDU3.zip já

In [44]:
# Célula 5: Extração e Renomeação dos Arquivos

# Configuração básica do logging
logging.basicConfig(filename='extracao_renomeacao.log', level=logging.ERROR)

# Função de extração e renomeação
def extrair_e_renomear_arquivo(papel):
    caminho_arquivo_zip = os.path.join(pasta_trimestre, f"{papel}.zip")
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    os.makedirs(pasta_balancos_trimestre, exist_ok=True)
    caminho_arquivo_xls = os.path.join(pasta_balancos_trimestre, "balanco.xls")
    novo_nome_arquivo = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")

    # Verifica se o arquivo já foi extraído e renomeado
    if os.path.exists(novo_nome_arquivo):
        print(f"Arquivo {novo_nome_arquivo} já foi extraído e renomeado.")
        return True  # Indica sucesso sem necessidade de renomeação

    try:
        # Extraindo o arquivo ZIP
        with zipfile.ZipFile(caminho_arquivo_zip, "r") as zip_ref:
            zip_ref.extractall(pasta_balancos_trimestre)

        # Renomeando o arquivo
        if os.path.exists(caminho_arquivo_xls):
            os.rename(caminho_arquivo_xls, novo_nome_arquivo)
            print(f"Arquivo {novo_nome_arquivo} extraído e renomeado com sucesso!")
        else:
            print(f"Arquivo balanco.xls não encontrado em {pasta_balancos_trimestre}")
            return False  # Indica falha na renomeação

    except Exception as e:
        logging.error(f"Erro ao extrair e renomear arquivo para {papel}: {e}")
        return False  # Indica falha na operação

    return True  # Indica sucesso

# Lista para armazenar papéis com erro
papeis_com_erro = []

# Extração e renomeação dos arquivos para cada empresa
for papel in lista_de_empresas[:]:  # Usar uma cópia da lista para evitar problemas ao remover elementos
    sucesso = extrair_e_renomear_arquivo(papel)
    if not sucesso:
        papeis_com_erro.append(papel)

# Remover os papéis com erro da lista de empresas
for papel in papeis_com_erro:
    lista_de_empresas.remove(papel)

# Exibir os papéis que deram erro
print("PAPÉIS COM ERRO:", papeis_com_erro)
print("LISTA DE EMPRESAS ATUALIZADA:", lista_de_empresas)
print("QUANTIDADE DE EMPRESAS ATUALIZADA:", len(lista_de_empresas))


Arquivo balancos\Q2-2024\AALR3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\ABCB3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\ABCB4.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\ABEV3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\ABYA3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\ACES3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\ACES4.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\ADHM3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AEDU11.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AEDU3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AELP3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AERI3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AESB3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AESL3.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AESL4.xls já foi extraído e renomeado.
Arquivo balancos\Q2-2024\AFLT3.xls já f

In [45]:
# Célula 6: Conversão de XLS para XLSX
def converter_xls_para_xlsx(papel):
    pasta_balancos_trimestre = os.path.join(BALANCOS_FOLDER, trimestre_anterior)
    pasta_balancos_xlsx_trimestre = os.path.join(BALANCOS_XLSX_FOLDER, trimestre_anterior)
    os.makedirs(pasta_balancos_xlsx_trimestre, exist_ok=True)

    caminho_xls = os.path.join(pasta_balancos_trimestre, f"{papel}.xls")
    caminho_xlsx = os.path.join(pasta_balancos_xlsx_trimestre, f"{papel}.xlsx")

    if os.path.isfile(caminho_xlsx):
        print(f"Arquivo {papel}.xlsx já foi convertido.")
        return

    try:
        xls = pd.ExcelFile(caminho_xls, engine="xlrd")
        with pd.ExcelWriter(caminho_xlsx, engine="openpyxl") as writer:
            for sheet_name in xls.sheet_names:
                df = pd.read_excel(xls, sheet_name=sheet_name)
                df.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"Arquivo {papel}.xls convertido para {papel}.xlsx.")
    except Exception as e:
        print(f"Erro ao converter {papel}.xls: {e}")

# Converter arquivos XLS para XLSX para cada empresa
for papel in lista_de_empresas:
    converter_xls_para_xlsx(papel)

Arquivo AALR3.xlsx já foi convertido.
Arquivo ABCB3.xlsx já foi convertido.
Arquivo ABCB4.xlsx já foi convertido.
Arquivo ABEV3.xlsx já foi convertido.
Arquivo ABYA3.xlsx já foi convertido.
Arquivo ACES3.xlsx já foi convertido.
Arquivo ACES4.xlsx já foi convertido.
Arquivo ADHM3.xlsx já foi convertido.
Arquivo AEDU11.xlsx já foi convertido.
Arquivo AEDU3.xlsx já foi convertido.
Arquivo AELP3.xlsx já foi convertido.
Arquivo AERI3.xlsx já foi convertido.
Arquivo AESB3.xlsx já foi convertido.
Arquivo AESL3.xlsx já foi convertido.
Arquivo AESL4.xlsx já foi convertido.
Arquivo AFLT3.xlsx já foi convertido.
Arquivo AFLU3.xlsx já foi convertido.
Arquivo AFLU5.xlsx já foi convertido.
Arquivo AGEI3.xlsx já foi convertido.
Arquivo AGEN33.xlsx já foi convertido.
Arquivo AGIN3.xlsx já foi convertido.
Arquivo AGRO3.xlsx já foi convertido.
Arquivo AGXY3.xlsx já foi convertido.
Arquivo AHEB3.xlsx já foi convertido.
Arquivo AHEB5.xlsx já foi convertido.
Arquivo AHEB6.xlsx já foi convertido.
Arquivo AL

In [46]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Constantes
BAL_PATRIM = "Bal. Patrim."
DEM_RESULT = "Dem. Result."
EXTENSAO_EXCEL = ".xlsx"


def reordenar_primeira_linha(df: pd.DataFrame) -> pd.DataFrame:
    """
    Reordena a primeira linha de um DataFrame, colocando as datas da mais antiga para a mais recente.
    """
    # Identificar a primeira linha (após remoção da original) para as datas
    primeira_linha = df.iloc[0, 1:]

    # Tentar converter os valores para datetime, forçando o formato "dd/mm/yyyy"
    datas = pd.to_datetime(primeira_linha, format="%d/%m/%Y", errors="coerce")

    # Verificar se todas as datas são válidas
    if datas.isnull().all():
        logging.warning(
            "Nenhuma data foi reconhecida na primeira linha. Retornando DataFrame original."
        )
        return df

    # Criar um DataFrame temporário para facilitar a reordenação
    temp_df = pd.DataFrame({"original_col": df.columns[1:], "date": datas})

    # Remover entradas que não são datas válidas
    temp_df = temp_df.dropna(subset=["date"])

    # Ordenar as colunas com base nas datas
    temp_df = temp_df.sort_values(by="date")

    # Reordenar o DataFrame original de acordo com as colunas ordenadas
    new_order = [df.columns[0]] + temp_df["original_col"].tolist()
    df = df[new_order]

    # Substituir a primeira linha com as datas ordenadas
    df.iloc[0, 1:] = temp_df["date"].dt.strftime("%d/%m/%Y").values

    return df


def processar_arquivo(caminho_arquivo: str, pasta_saida: str) -> None:
    try:
        nome_arquivo = os.path.basename(caminho_arquivo)
        nome_papel = nome_arquivo.replace(EXTENSAO_EXCEL, "")

        bal_patrim = pd.read_excel(
            caminho_arquivo, sheet_name=BAL_PATRIM, header=None, engine="openpyxl"
        )
        dem_result = pd.read_excel(
            caminho_arquivo, sheet_name=DEM_RESULT, header=None, engine="openpyxl"
        )

        # Remover a primeira linha de ambas as planilhas
        bal_patrim = bal_patrim.iloc[1:].reset_index(drop=True)
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Reordenar a primeira linha de ambas as planilhas
        bal_patrim = reordenar_primeira_linha(bal_patrim)
        dem_result = reordenar_primeira_linha(dem_result)

        # Verificar o conteúdo da célula [0, 0] da planilha 'Bal. Patrim.'
        primeira_celula = bal_patrim.iloc[0, 0]

        if (
            pd.to_datetime(primeira_celula, format="%d/%m/%Y", errors="coerce")
            is not pd.NaT
        ):
            # Se a célula contém uma data, mover as colunas para a direita
            bal_patrim = bal_patrim.shift(1, axis=1)

        # Definir o nome do papel na célula [0, 0] da planilha 'Bal. Patrim.'
        bal_patrim.iloc[0, 0] = nome_papel

        # Remover a primeira linha de 'Dem. Result.' antes de concatenar
        dem_result = dem_result.iloc[1:].reset_index(drop=True)

        # Concatenar as duas planilhas
        combinado = pd.concat([bal_patrim, dem_result], ignore_index=True)
        
        # Verificação final: Se a célula [0, 0] não contiver o nome do papel, remover a primeira linha
        if combinado.iloc[0, 0] != nome_papel:
            combinado = combinado.iloc[1:].reset_index(drop=True)

        # Criar a pasta de saída correspondente
        nome_subpasta = os.path.basename(os.path.dirname(caminho_arquivo))
        subpasta_saida = os.path.join(pasta_saida, nome_subpasta)
        os.makedirs(subpasta_saida, exist_ok=True)

        # Criar o caminho para o arquivo de saída
        arquivo_saida = os.path.join(subpasta_saida, nome_arquivo)

        # Verificar se o arquivo já existe antes de salvar
        if os.path.exists(arquivo_saida):
            logging.info(f"Arquivo já existe e será ignorado: {arquivo_saida}")
        else:
            # Salvar o resultado em uma nova planilha
            combinado.to_excel(arquivo_saida, index=False)
            logging.info(f"Arquivo combinado e ordenado salvo como: {arquivo_saida}")

    except Exception as e:
        logging.error(f"Erro ao processar o arquivo {caminho_arquivo}: {str(e)}")


def processar_todos_arquivos(pasta_base: str, pasta_saida: str) -> None:
    for nome_subpasta in sorted(os.listdir(pasta_base)):
        caminho_subpasta = os.path.join(pasta_base, nome_subpasta)
        if os.path.isdir(caminho_subpasta):
            for nome_arquivo in sorted(os.listdir(caminho_subpasta)):
                if nome_arquivo.endswith(EXTENSAO_EXCEL):
                    caminho_arquivo = os.path.join(caminho_subpasta, nome_arquivo)
                    processar_arquivo(caminho_arquivo, pasta_saida)


# Execução do processamento
processar_todos_arquivos(BALANCOS_XLSX_FOLDER, BALANCOS_CONCATENADOS_FOLDER)


2024-08-24 23:52:50,927 - INFO - Arquivo já existe e será ignorado: balancos_concatenados\Q1-2021\AALR3.xlsx
2024-08-24 23:52:51,151 - INFO - Arquivo já existe e será ignorado: balancos_concatenados\Q1-2021\ABCB3.xlsx
2024-08-24 23:52:51,324 - INFO - Arquivo já existe e será ignorado: balancos_concatenados\Q1-2021\ABCB4.xlsx
2024-08-24 23:52:51,538 - INFO - Arquivo já existe e será ignorado: balancos_concatenados\Q1-2021\ABEV3.xlsx
C:\Users\jvcca\AppData\Local\Temp\ipykernel_16020\3786195548.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[0, 1:] = temp_df["date"].dt.strftime("%d/%m/%Y").values
2024-08-24 23:52:51,725 - INFO - Arquivo já existe e será ignorado: balancos_concatenados\Q1-2021\ABYA3.xlsx
C:\Users\jvcca\AppData\Local\Temp\ipykernel_16020\3786195548.py:43: SettingWithCopyWarning

In [11]:
# Função para obter as subpastas ordenadas da mais antiga para a mais recente
def obter_subpastas_ordenadas(base_folder):
    subpastas = [f.name for f in os.scandir(base_folder) if f.is_dir()]
    subpastas.sort(
        key=lambda x: datetime.strptime(x.split("-")[1], "%Y")
    )  # Ordenar por ano
    return subpastas

# Função para concatenar as planilhas
def concatenar_planilhas(base_folder, output_folder):
    planilhas_dict = {}

    # Obter subpastas ordenadas
    subpastas_ordenadas = obter_subpastas_ordenadas(base_folder)
    
    # Percorre todas as subpastas (ordenadas pela data da pasta mais antiga para a mais recente)
    for subpasta in subpastas_ordenadas:
        subpasta_path = os.path.join(base_folder, subpasta)
        if os.path.isdir(subpasta_path):
            for file_name in os.listdir(subpasta_path):
                if file_name.endswith(".xlsx"):
                    file_path = os.path.join(subpasta_path, file_name)

                    # Carregar a planilha, ignorando a primeira linha (índice de colunas)
                    df = pd.read_excel(file_path, skiprows=1)

                    # Se o arquivo já foi processado, concatenar as novas colunas
                    if file_name in planilhas_dict:
                        df_existente = planilhas_dict[file_name]

                        # Ignorar a primeira coluna (indicadores financeiros)
                        for col in df.columns[1:]:
                            if col not in df_existente.columns:
                                df_existente[col] = df[col]

                        # Reordenar as colunas pela data
                        colunas_datas = [
                            col
                            for col in df_existente.columns
                            if col != df_existente.columns[0]
                        ]
                        colunas_datas.sort(
                            key=lambda date: datetime.strptime(str(date), "%d/%m/%Y")
                        )
                        colunas_ordenadas = [df_existente.columns[0]] + colunas_datas
                        planilhas_dict[file_name] = df_existente[colunas_ordenadas]
                    else:
                        # Armazenar a primeira planilha encontrada
                        planilhas_dict[file_name] = df

    # Salvar os arquivos concatenados na pasta de destino
    for nome_arquivo, df_final in planilhas_dict.items():
        # Salvar o DataFrame concatenado no arquivo de destino
        output_path = os.path.join(output_folder, nome_arquivo)
        df_final.to_excel(output_path, index=False)

# Executar a função
concatenar_planilhas(BALANCOS_CONCATENADOS_FOLDER, BALANCOS_DEFINITIVO_FOLDER)


In [81]:
# Dicionário para armazenar os DataFrames
dicionario_de_balancos = {}
contador_de_empresas = 0
# Percorre todos os arquivos na pasta 'balancos_definitivos'
for arquivo in os.listdir(BALANCOS_DEFINITIVO_FOLDER):
    if arquivo.endswith(".xlsx"):
        # Cria o caminho completo para o arquivo
        caminho_arquivo = os.path.join(BALANCOS_DEFINITIVO_FOLDER, arquivo)
        contador_de_empresas = contador_de_empresas + 1
        # Carrega o DataFrame a partir do arquivo Excel
        balanco = pd.read_excel(caminho_arquivo, sheet_name=0, engine="openpyxl", header=None)   

        # Na célula [0, 0] está o nome da empresa
        nome = balanco.iloc[0, 0]
        print(f"Processando empresa {nome}...")
        # Pega a 1ª linha e torna um cabeçalho
        balanco.columns = balanco.iloc[0]
        balanco = balanco[1:]

        # Define a 1ª coluna (nome da empresa) como índice
        balanco = balanco.set_index(nome)

        # Adiciona o DataFrame ao dicionário com o nome da empresa como chave
        dicionario_de_balancos[nome] = balanco
print("TOTAL DE EMPRESAS: ", contador_de_empresas)

Processando empresa AALR3...
Processando empresa ABCB3...
Processando empresa ABCB4...
Processando empresa ABEV3...
Processando empresa ABYA3...
Processando empresa ACES3...
Processando empresa ACES4...
Processando empresa ADHM3...
Processando empresa AEDU11...
Processando empresa AEDU3...
Processando empresa AELP3...
Processando empresa AERI3...
Processando empresa AESB3...
Processando empresa AESL3...
Processando empresa AESL4...
Processando empresa AFLT3...
Processando empresa AFLU3...
Processando empresa AFLU5...
Processando empresa AGEI3...
Processando empresa AGEN33...
Processando empresa AGIN3...
Processando empresa AGRO3...
Processando empresa AGXY3...
Processando empresa AHEB3...
Processando empresa AHEB5...
Processando empresa AHEB6...
Processando empresa ALBA3...
Processando empresa ALLD3...
Processando empresa ALLL11...
Processando empresa ALLL3...
Processando empresa ALLL4...
Processando empresa ALOS3...
Processando empresa ALPA3...
Processando empresa ALPA4...
Processando

In [82]:
# Conjunto de nomes no dicionário
nomes_no_dicionario = set(dicionario_de_balancos.keys())

# Conjunto de nomes na lista de empresas
nomes_na_lista = set(lista_de_empresas)

# Empresas que estão no dicionário mas não na lista de empresas
faltando_na_lista = nomes_no_dicionario - nomes_na_lista

# Atualizar lista_de_empresas para incluir as empresas que estão no dicionário
lista_de_empresas.extend(faltando_na_lista)

# Remover duplicatas da lista_de_empresas (caso haja)
lista_de_empresas = list(set(lista_de_empresas))
sorted_lista_de_empresas = sorted(lista_de_empresas)
print("LISTA ATUALIZADA DE EMPRESAS:")
print(sorted_lista_de_empresas)

# Opcional: Exibir as empresas que estavam no dicionário, mas não na lista antes da atualização
print("Empresas que foram adicionadas à lista:")
print(faltando_na_lista)

LISTA ATUALIZADA DE EMPRESAS:
['AALR3', 'ABCB3', 'ABCB4', 'ABEV3', 'ABYA3', 'ACES3', 'ACES4', 'ADHM3', 'AEDU11', 'AEDU3', 'AELP3', 'AERI3', 'AESB3', 'AESL3', 'AESL4', 'AFLT3', 'AFLU3', 'AFLU5', 'AGEI3', 'AGEN33', 'AGIN3', 'AGRO3', 'AGXY3', 'AHEB3', 'AHEB5', 'AHEB6', 'ALBA3', 'ALLD3', 'ALLL11', 'ALLL3', 'ALLL4', 'ALOS3', 'ALPA3', 'ALPA4', 'ALPK3', 'ALSC3', 'ALSO3', 'ALUP11', 'ALUP3', 'ALUP4', 'AMAR3', 'AMBP3', 'AMBV3', 'AMBV4', 'AMER3', 'AMIL3', 'AMPI3', 'ANIM3', 'APER3', 'APTI4', 'ARCE3', 'ARCZ3', 'ARCZ6', 'ARLA3', 'ARLA4', 'ARML3', 'ARPS3', 'ARPS4', 'ARTE3', 'ARTE4', 'ARTR3', 'ARZZ3', 'ASAI3', 'ASSM3', 'ASSM4', 'ASTA4', 'ATMP3', 'ATOM3', 'AURA33', 'AURE3', 'AUTM3', 'AVIL3', 'AVLL3', 'AZEV3', 'AZEV4', 'AZUL4', 'AZZA3', 'B3SA3', 'BAHI11', 'BAHI3', 'BAHI4', 'BAHI5', 'BALM3', 'BALM4', 'BAUH4', 'BAZA3', 'BBAS3', 'BBDC3', 'BBDC4', 'BBRK3', 'BBSE3', 'BBTG11', 'BBTG12', 'BBTG13', 'BCAL6', 'BDLL3', 'BDLL4', 'BECE3', 'BECE4', 'BEEF3', 'BEES3', 'BEES4', 'BELG3', 'BELG4', 'BEMA3', 'BERG3', 'BESP3

In [88]:
dicionario_de_balancos["AALR3"]

,31/12/2015,31/03/2016,30/06/2016,30/09/2016,31/12/2016,31/03/2017,30/06/2017,30/09/2017,31/12/2017,31/03/2018,...,31/03/2022,30/06/2022,30/09/2022,31/12/2022,31/03/2023,30/06/2023,30/09/2023,31/12/2023,31/03/2024,30/06/2024
AALR3,,,,,,,,,,,,,,,,,,,,,
Ativo Total,1536352,1844579.968,1872144,1934174.976,2166311.936,2218670.08,2270885.12,2308239.104,2284985.088,2274482.944,...,2537389.056,2573832.96,2631220.992,2615453.952,2571685.12,2520962.048,2748189.952,2669810.944,2573677.056,2692409.088
Ativo Circulante,215926,280523.008,292211.008,347318.016,494428.992,384152.992,409799.008,430387.008,401553.984,399984.992,...,456177.984,457827.008,508798.016,484284,433144,386719.008,620451.968,524747.008,441300.992,563769.024
Caixa e Equivalentes de Caixa,21920,25183,23851,44720,159332.992,53505,51155,50806,91597,68523,...,81625,240190,247828.992,218744,154614,95520,294614.016,218595.008,97788,254478
Aplicações Financeiras,3672,12637,13593,1856,37811,0,0,0,10,0,...,0,0,0,0,0,0,0,0,0,0
Contas a Receber,155592,199127.008,208658,233011.008,233260,249166,263584,282444,219178,235863.008,...,275668.992,133735,171106,186219.008,189298,206091.008,236254,202160,234380,186158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IR Diferido,NaN,0,0,0,18341,2541,-2431,37,45127,1315,...,14736,18174,2793,2848,0,-2974,-2820,12220,1198,0
Participações/Contribuições Estatutárias,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Reversão dos Juros sobre Capital Próprio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [86]:
# Configurações
data_referencia = "01/01/2023"
data_referencia_aux = pd.to_datetime(data_referencia, format="%d/%m/%Y")

# Lista para armazenar empresas a serem removidas
lista_para_remover = []

# Filtra empresas com último balanço anterior à data de referência
for papel, balanco in dicionario_de_balancos.items():
    colunas = list(balanco.columns)
    
    try:
        # Converte a última coluna para datetime, especificando o formato e dayfirst=True
        ultima_data_aux = pd.to_datetime(colunas[-1], format="%d/%m/%Y", dayfirst=True)
        
        if ultima_data_aux <= data_referencia_aux:
            lista_para_remover.append(papel)
    except Exception as e:
        print(f"Erro ao processar a data para {papel}: {e}")

# Exibe a lista de empresas que serão removidas
print("LISTA DE EMPRESAS A SEREM REMOVIDAS: ", lista_para_remover)

# Remove as empresas do dicionário
for papel in lista_para_remover:
    dicionario_de_balancos.pop(papel)

# Exibe o total remanescente e a lista atualizada de empresas
print("\nTOTAL REMANESCENTE:", len(dicionario_de_balancos.keys()))


LISTA DE EMPRESAS A SEREM REMOVIDAS:  ['ABYA3', 'ACES3', 'ACES4', 'ADHM3', 'AEDU11', 'AEDU3', 'AELP3', 'AFLU3', 'AFLU5', 'AGEI3', 'AGEN33', 'AGIN3', 'ALBA3', 'ALSC3', 'ALSO3', 'AMBV3', 'AMBV4', 'AMIL3', 'AMPI3', 'ARCE3', 'ARCZ3', 'ARCZ6', 'ARLA3', 'ARLA4', 'ARPS3', 'ARPS4', 'ARTE3', 'ARTE4', 'ARZZ3', 'ASSM3', 'ASSM4', 'ASTA4', 'AUTM3', 'AVIL3', 'BBRK3', 'BBSE3', 'BBTG11', 'BCAL6', 'BECE3', 'BECE4', 'BELG3', 'BELG4', 'BEMA3', 'BERG3', 'BESP3', 'BESP4', 'BFIT3', 'BFIT4', 'BHGR3', 'BICB3', 'BICB4', 'BIDI11', 'BIDI3', 'BIDI4', 'BISA3', 'BKBR3', 'BLUT3', 'BLUT4', 'BMEF3', 'BMTO3', 'BMTO4', 'BNCA3', 'BOVH3', 'BPAT33', 'BPHA3', 'BPIA3', 'BRDT3', 'BRTP3', 'BRTP4', 'BSCT3', 'BSCT5', 'BSCT6', 'BSEV3', 'BTTL3', 'BUET3', 'BUET4', 'CAFE3', 'CAFE4', 'CARD3', 'CCHI3', 'CCHI4', 'CCIM3', 'CCPR3', 'CCTU4', 'CCXC3', 'CELM3', 'CFLU4', 'CGOS3', 'CGOS4', 'CIQU3', 'CIQU4', 'CLAN3', 'CLAN4', 'CMET4', 'CMGI3', 'CMGI4', 'CMMA4', 'CNFB4', 'CPNY3', 'CPSL3', 'CRBM3', 'CRBM7', 'CRDE3', 'CREM3', 'CREM4', 'CRTP3', 'C

In [92]:
import yfinance as yf

# Dicionário para armazenar as cotações
dicionario_de_cotacoes = {}
tickers_falhados = []
keys_list = list(dicionario_de_balancos.keys())
# Itera sobre as chaves do dicionário de balanços
for papel in keys_list:
    # Adiciona ".SA" ao nome do papel para buscar na B3
    papel_com_sufixo = papel + ".SA"
    
    try:
        print(f"Obtendo cotações para {papel_com_sufixo}...")
        
        # Obtém os dados de mercado usando o Yahoo Finance
        cotacao = yf.download(papel_com_sufixo)
        
        if not cotacao.empty:
            # Seleciona as colunas desejadas e organiza o DataFrame
            cotacao_filtrada = cotacao[['Open', 'High', 'Low', 'Close', 'Adj Close']]
            cotacao_filtrada.columns = ['Abertura', 'Máxima', 'Baixa', 'Fechamento', 'Fechamento Ajustado']
            
            # Adiciona o DataFrame ao dicionário de cotações
            dicionario_de_cotacoes[papel] = cotacao_filtrada
        else:
            print(f"Cotações não encontradas para {papel_com_sufixo}")
            tickers_falhados.append(papel_com_sufixo)

    except Exception as e:
        print(f"Erro ao baixar cotações para {papel_com_sufixo}: {e}")
        tickers_falhados.append(papel_com_sufixo)

# Exibe o resultado
print("\nTOTAL DE COTAÇÕES OBTIDAS:", len(dicionario_de_cotacoes.keys()))
print("\nCOTAÇÕES OBTIDAS PARA OS SEGUINTES TICKERS:", dicionario_de_cotacoes.keys())
print("COTAÇÕES NÃO ENCONTRADAS PARA OS SEGUINTES TICKERS:", tickers_falhados)

Obtendo cotações para AALR3.SA...


[*********************100%%**********************]  1 of 1 completed


Obtendo cotações para ABCB3.SA...


[*********************100%%**********************]  1 of 1 completed
2024-08-25 17:42:46,012 - ERROR - 
1 Failed download:
2024-08-25 17:42:46,014 - ERROR - ['ABCB3.SA']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 1925-09-19 -> 2024-08-25)')


$ABCB3.SA: possibly delisted; No price data found  (1d 1925-09-19 -> 2024-08-25)
Cotações não encontradas para ABCB3.SA
Obtendo cotações para ABCB4.SA...


...

Obtendo cotações para YDUQ3.SA...


[*********************100%%**********************]  1 of 1 completed


Obtendo cotações para ZAMP3.SA...


[*********************100%%**********************]  1 of 1 completed


TOTAL DE COTAÇÕES OBTIDAS: 479
COTAÇÕES NÃO ENCONTRADAS PARA OS SEGUINTES TICKERS: ['ABCB3.SA', 'AESL3.SA', 'AESL4.SA', 'ALLL11.SA', 'ALLL3.SA', 'ALLL4.SA', 'ARTR3.SA', 'BAHI11.SA', 'BAHI4.SA', 'BAHI5.SA', 'BBTG12.SA', 'BBTG13.SA', 'BIOM4.SA', 'BMGB11.SA', 'BNBR4.SA', 'BOAS3.SA', 'BPNM3.SA', 'BRML3.SA', 'BRSR4.SA', 'CALI4.SA', 'CAMB4.SA', 'CEPE3.SA', 'CEPE5.SA', 'CEPE6.SA', 'CESP3.SA', 'CESP4.SA', 'CESP5.SA', 'CESP6.SA', 'CLSC5.SA', 'CLSC6.SA', 'CPFG3.SA', 'CPFG4.SA', 'CPFP4.SA', 'CPLE11.SA', 'CPRE3.SA', 'CTSA8.SA', 'CYRE4.SA', 'EBEN4.SA', 'EEEL3.SA', 'EEEL4.SA', 'ELCA3.SA', 'ELCA4.SA', 'ELPL3.SA', 'ELPL5.SA', 'ELPL6.SA', 'ENBR3.SA', 'ENER3.SA', 'ENER5.SA', 'ENER6.SA', 'EPAR4.SA', 'EQMA5B.SA', 'EQMA6B.SA', 'ESCE3.SA', 'ESTC11.SA', 'ESTC4.SA', 'ETER4.SA', 'FLCL3.SA', 'FLCL5.SA', 'FLCL6.SA', 'FRAS4.SA', 'FRTA3.SA', 'GLOB4.SA', 'GUAR4.SA', 'IGTA3.SA', 'INHA3.SA', 'JPSA3.SA', 'KROT4.SA', 'LCAM3.SA', 'LEVE4.SA', 'LIGH3.SA', 'LREN4.SA', 'MEND5.SA', 'MEND6.SA', 'MLFT4.SA', 'MNDL4.SA', 'MNPR4

In [100]:
# Obtém as chaves de ambos os dicionários
chaves_balancos = set(dicionario_de_balancos.keys())
chaves_cotacoes = set(dicionario_de_cotacoes.keys())

# Encontra as chaves comuns
chaves_comuns = chaves_balancos.intersection(chaves_cotacoes)

# Remove as entradas dos dicionários que não estão nas chaves comuns
for chave in list(dicionario_de_balancos.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_balancos[chave]

for chave in list(dicionario_de_cotacoes.keys()):
    if chave not in chaves_comuns:
        del dicionario_de_cotacoes[chave]

print("Dicionários atualizados!")
print(f"Total de empresas restantes nos balanços: {len(dicionario_de_balancos.keys())}")
print(f"Total de empresas restantes nas cotações: {len(dicionario_de_cotacoes.keys())}")

Dicionários atualizados!
Total de empresas restantes nos balanços: 479
Total de empresas restantes nas cotações: 479
